In [31]:
from docopt import docopt
from glob import glob
import os
import subprocess
import pandas as pd
from tqdm import tqdm
import json

In [23]:
cd ~/work/ocaml-workspace/symetric

/home/feser/work/ocaml-workspace/symetric


In [79]:
csg_run = 'runs/llm-csg-numeric'

In [80]:
rows = []
for out in tqdm(glob(csg_run + "/*.out")):
    name_parts = os.path.basename(out).split(".")[0].split("-")
    bench_name = name_parts[1]
    repeat = name_parts[2]
    bench_file = glob(f"bench/cad_ext/**/{bench_name}", recursive=True)[0]
    proc = subprocess.run(["_build/default/bin/equiv_cad.exe", bench_file, out])
    rows.append({"bench": bench_name, "repeat": repeat, "result": proc.returncode})
csg_df = pd.DataFrame(rows)

100%|█████████████████████████████████████████████████████████████████████████████████| 400/400 [00:05<00:00, 76.75it/s]


In [81]:
csg_df_grouped = csg_df.groupby('bench').agg('min')

In [82]:
def print_csg_results(df):
    print('Correct: ', len(df[df['result'] == 0]))
    print('Incorrect: ', len(df[df['result'] == 1]))
    print('Invalid sexp: ', len(df[df['result'] == 2]))
    print('Invalid CSG program: ', len(df[df['result'] == 3]))
    
print_csg_results(csg_df_grouped)

Correct:  0
Incorrect:  40
Invalid sexp:  0
Invalid CSG program:  0


In [74]:
regex_run = 'runs/llm-regex-nosketch-nonl/'

In [75]:
rows = []
for out in tqdm(glob(regex_run + "/*.out")):
    name_parts = os.path.basename(out).split(".")[0].split("-")
    bench_name = name_parts[1]
    bench_file = f"vendor/regel/exp/so/benchmark/{bench_name}"
    
    try:
        with open(out, 'r') as out_f:
            for (repeat, program) in enumerate(json.load(out_f)):
                proc = subprocess.run(["_build/default/bin/check_regex.exe", program], stdin=open(bench_file))
                rows.append({"bench": bench_name, "repeat": repeat, "result": proc.returncode})
    except json.JSONDecodeError:
        print('Warning: Could not read ', out)
regex_df = pd.DataFrame(rows)


 40%|████████████████████████████████▉                                                 | 49/122 [00:05<00:06, 11.53it/s]

 47%|██████████████████████████████████████▎                                           | 57/122 [00:05<00:05, 11.22it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 122/122 [00:12<00:00,  9.41it/s]


In [76]:
regex_grouped_df = regex_df.groupby('bench').agg('min')
regex_grouped_df

,repeat,result
bench,,
1,0,0
10,0,1
100,0,1
101,0,1
102,0,1
...,...,...
95,0,1
96,0,0
97,0,2


In [77]:
def print_results(df):
    print('Correct: ', len(df[df['result'] == 0]))
    print('Incorrect: ', len(df[df['result'] == 1]))
    print('Invalid regex program: ', len(df[df['result'] == 2]))
    
print_results(regex_grouped_df)

Correct:  15
Incorrect:  58
Invalid regex program:  47
